In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor


In [3]:
df = pd.read_csv("/Users/macbook/Desktop/amazon_stock_data_XGBoost-and-KNN/Amazon stock data 2000-2025.csv")
df.head()

,date,open,high,low,close,adj_close,volume
0,2000-01-03 00:00:00-05:00,4.075000,4.478125,3.952344,4.468750,4.468750,322352000
1,2000-01-04 00:00:00-05:00,4.268750,4.575000,4.087500,4.096875,4.096875,349748000
2,2000-01-05 00:00:00-05:00,3.525000,3.756250,3.400000,3.487500,3.487500,769148000
3,2000-01-06 00:00:00-05:00,3.565625,3.634375,3.200000,3.278125,3.278125,375040000
4,2000-01-07 00:00:00-05:00,3.350000,3.525000,3.309375,3.478125,3.478125,210108000


In [4]:
df['adj_close'].max()

242.05999755859372

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6321 entries, 0 to 6320
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date       6321 non-null   object 
 1   open       6321 non-null   float64
 2   high       6321 non-null   float64
 3   low        6321 non-null   float64
 4   close      6321 non-null   float64
 5   adj_close  6321 non-null   float64
 6   volume     6321 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 345.8+ KB


In [6]:
df.describe()

,open,high,low,close,adj_close,volume
count,6321.000000,6321.000000,6321.000000,6321.000000,6321.000000,6.321000e+03
mean,44.736023,45.251977,44.176568,44.729400,44.729400,1.180771e+08
std,59.578629,60.241809,58.852822,59.562415,59.562415,9.758263e+07
min,0.295500,0.305000,0.275500,0.298500,0.298500,1.500750e+07
25%,2.363500,2.415500,2.306000,2.364000,2.364000,6.215930e+07
50%,11.602000,11.737500,11.420000,11.595000,11.595000,9.595200e+07
75%,85.140503,85.779999,83.903000,84.817497,84.817497,1.433920e+08
max,239.020004,242.520004,238.029999,242.059998,242.059998,2.086584e+09


In [7]:
df.isnull().sum()

date         0
open         0
high         0
low          0
close        0
adj_close    0
volume       0
dtype: int64

In [8]:
df = df.drop('date', axis=1)
df.head()

,open,high,low,close,adj_close,volume
0,4.075000,4.478125,3.952344,4.468750,4.468750,322352000
1,4.268750,4.575000,4.087500,4.096875,4.096875,349748000
2,3.525000,3.756250,3.400000,3.487500,3.487500,769148000
3,3.565625,3.634375,3.200000,3.278125,3.278125,375040000
4,3.350000,3.525000,3.309375,3.478125,3.478125,210108000


In [9]:
x = df.drop('volume', axis=1)
y = df['volume']

In [10]:
x

,open,high,low,close,adj_close
0,4.075000,4.478125,3.952344,4.468750,4.468750
1,4.268750,4.575000,4.087500,4.096875,4.096875
2,3.525000,3.756250,3.400000,3.487500,3.487500
3,3.565625,3.634375,3.200000,3.278125,3.278125
4,3.350000,3.525000,3.309375,3.478125,3.478125
...,...,...,...,...,...
6316,230.460007,231.179993,228.160004,228.929993,228.929993
6317,228.850006,230.419998,227.520004,230.369995,230.369995
6318,229.199997,229.889999,227.229996,228.679993,228.679993
6319,228.820007,229.300003,223.720001,226.649994,226.649994


In [11]:
y

0       322352000
1       349748000
2       769148000
3       375040000
4       210108000
          ...    
6316     32285200
6317     31346500
6318     27031100
6319     42975100
6320     28566700
Name: volume, Length: 6321, dtype: int64

In [12]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [13]:
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [14]:
x_train

array([[-0.70018157, -0.70043521, -0.70074718, -0.70070255, -0.70070255],
       [ 2.34794223,  2.34388   ,  2.36224699,  2.3665696 ,  2.3665696 ],
       [-0.65050657, -0.65055325, -0.65056446, -0.65192078, -0.65192078],
       ...,
       [ 1.93335571,  1.96853531,  1.95589397,  1.99729368,  1.99729368],
       [ 1.92274399,  1.90146011,  1.93416801,  1.92306153,  1.92306153],
       [-0.72630471, -0.72682966, -0.7268049 , -0.72692328, -0.72692328]])

In [15]:
x_test

array([[-0.26927124, -0.26189965, -0.26615799, -0.25779757, -0.25779757],
       [-0.43381602, -0.43449472, -0.42971809, -0.43183754, -0.43183754],
       [-0.73893494, -0.73897095, -0.7390878 , -0.73940136, -0.73940136],
       ...,
       [-0.66423399, -0.66205683, -0.66320338, -0.66362841, -0.66362841],
       [ 0.37216534,  0.40080243,  0.35307306,  0.37037759,  0.37037759],
       [-0.72302989, -0.72320219, -0.72275298, -0.72318823, -0.72318823]])

# XGBoost Regressor

In [16]:
xgb_reg = xgb.XGBRegressor(objective ='reg:squarederror')
xgb_reg.fit(x_train, y_train)



XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [17]:
y_pred = xgb_reg.predict(x_test)
y_pred

array([1.2763510e+08, 5.6179268e+07, 1.4049072e+08, ..., 1.9034224e+08,
       1.1477901e+08, 1.3388914e+08], dtype=float32)

In [18]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Assuming y_test and y_pred are your true and predicted values respectively
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R-squared: {r2}")

Mean Squared Error: 5700470503640863.0
Mean Absolute Error: 41645184.84110672
R-squared: 0.2284288975205463


In [19]:
from sklearn.model_selection import GridSearchCV

In [20]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 0.9, 1.0]
}

grid_search = GridSearchCV(estimator=xgb.XGBRegressor(objective='reg:squarederror'), param_grid=param_grid, cv=3, scoring='r2', verbose=2, n_jobs=-1)
grid_search.fit(x_train, y_train)

print(f"Best parameters: {grid_search.best_params_}")
print(f"Best R-squared: {grid_search.best_score_}")

# Train the model with the best parameters
best_xgb_reg = grid_search.best_estimator_
best_xgb_reg.fit(x_train, y_train)
y_pred = best_xgb_reg.predict(x_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R-squared: {r2}")

Fitting 3 folds for each of 81 candidates, totalling 243 fits
[CV] END learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.9; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=100, subsample=1.0; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.9; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.9; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=100, subsample=1.0; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=100, subsample=1.0; total time=   0.2s
[CV] END learning_rate=0.01, max_depth=3, n_estimators=200, subsample=0.8; total time=   0.3

In [21]:
import pickle
with open("XG_model.pkl",'wb') as file:
    pickle.dump((best_xgb_reg),file)

In [22]:
from sklearn.neighbors import KNeighborsRegressor


In [23]:
x

,open,high,low,close,adj_close
0,4.075000,4.478125,3.952344,4.468750,4.468750
1,4.268750,4.575000,4.087500,4.096875,4.096875
2,3.525000,3.756250,3.400000,3.487500,3.487500
3,3.565625,3.634375,3.200000,3.278125,3.278125
4,3.350000,3.525000,3.309375,3.478125,3.478125
...,...,...,...,...,...
6316,230.460007,231.179993,228.160004,228.929993,228.929993
6317,228.850006,230.419998,227.520004,230.369995,230.369995
6318,229.199997,229.889999,227.229996,228.679993,228.679993
6319,228.820007,229.300003,223.720001,226.649994,226.649994


In [24]:
y

0       322352000
1       349748000
2       769148000
3       375040000
4       210108000
          ...    
6316     32285200
6317     31346500
6318     27031100
6319     42975100
6320     28566700
Name: volume, Length: 6321, dtype: int64

In [25]:
knn = KNeighborsRegressor(n_neighbors=5)

In [26]:
knn.fit(x_train, y_train)


KNeighborsRegressor()

In [27]:
y_pred = knn.predict(x_test)
y_pred

array([1.290480e+08, 5.053000e+07, 1.611016e+08, ..., 3.656716e+08,
       1.400456e+08, 1.462072e+08])

In [28]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R-squared: {r2}")

Mean Squared Error: 5145538090887722.0
Mean Absolute Error: 39952541.48616601
R-squared: 0.3035402086370681


In [29]:
param ={
    'n_neighbors' : np.arange(1,100)
}

In [30]:
Grid_search = GridSearchCV(KNeighborsRegressor(), param, cv=5, n_jobs=1, verbose=5)

In [31]:
Grid_search.fit(x_train, y_train)

Fitting 5 folds for each of 99 candidates, totalling 495 fits
[CV 1/5] END ....................n_neighbors=1;, score=-0.658 total time=   0.0s
[CV 2/5] END ....................n_neighbors=1;, score=-0.015 total time=   0.0s
[CV 3/5] END ....................n_neighbors=1;, score=-0.096 total time=   0.0s
[CV 4/5] END ....................n_neighbors=1;, score=-0.253 total time=   0.0s
[CV 5/5] END .....................n_neighbors=1;, score=0.185 total time=   0.0s
[CV 1/5] END .....................n_neighbors=2;, score=0.079 total time=   0.0s
[CV 2/5] END .....................n_neighbors=2;, score=0.165 total time=   0.0s
[CV 3/5] END .....................n_neighbors=2;, score=0.117 total time=   0.0s
[CV 4/5] END .....................n_neighbors=2;, score=0.078 total time=   0.0s
[CV 5/5] END .....................n_neighbors=2;, score=0.238 total time=   0.0s
[CV 1/5] END .....................n_neighbors=3;, score=0.183 total time=   0.0s
[CV 2/5] END .....................n_neighbors=3

GridSearchCV(cv=5, estimator=KNeighborsRegressor(), n_jobs=1,
             param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])},
             verbose=5)

In [32]:
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best R-squared: {grid_search.best_score_}")

# Train the model with the best parameters
best_knn_reg = grid_search.best_estimator_
best_knn_reg.fit(x_train, y_train)
y_pred = best_knn_reg.predict(x_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R-squared: {r2}")

Best parameters: {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 300, 'subsample': 0.8}
Best R-squared: 0.2719960605382518
Mean Squared Error: 4758340582957261.0
Mean Absolute Error: 41196358.69090909
R-squared: 0.3559482349360187


In [33]:
import pickle
with open("KNN_model.pkl",'wb') as file:
    pickle.dump((best_knn_reg),file)